In [ ]:
from pasteur.kedro.ipython import *

register_kedro()


In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
from pasteur.metadata import Metadata
from pasteur.transform import TableTransformer, Attributes, get_dtype

view = "mimic_tab_admissions"
trn: TableTransformer = catalog.load(f"{view}.trn.table")
table: pd.DataFrame = catalog.load(f"{view}.wrk.idx_table")


In [ ]:
# sensitive
table.head()


In [ ]:
attrs = trn["idx"].get_attributes()


In [ ]:
# from pasteur.synth.math import expand_table, calc_marginal, calc_marginal_1way, AttrSelector, AttrSelectors
# cols, cols_noncommon, domains = expand_table(attrs, table)
from functools import reduce


In [ ]:
from typing import Callable

col_idx = {}
groups = []
heights = []
common = []
domains = []
for i, a in enumerate(attrs.values()):
    for n, c in a.cols.items():
        col_idx[n] = i
        groups.append(i)
        heights.append(c.lvl.height)
        common.append(a.common)
        domains.append([c.get_domain(h) for h in range(c.height)])

n = len(col_idx)
empty_pset = tuple(-1 for _ in range(n))


def add_to_pset(s, x, h):
    s = list(s)
    s[x] = h
    return tuple(s)


In [ ]:
def maximal_parents(V: tuple[int], tau: float, _pgroups=set()) -> list[tuple[int]]:
    if tau < 1:
        return []
    if not V:
        return [empty_pset]
    x = V[0]
    V = V[1:]

    S = []
    U = set()

    for h in range(heights[x]):
        dom = domains[x][h]
        g = groups[x]
        if g in _pgroups:
            dom -= common[x]
            _pgroups_with_x = _pgroups
        else:
            _pgroups_with_x = _pgroups.copy()
            _pgroups_with_x.add(g)

        for z in maximal_parents(V, tau / dom, _pgroups_with_x):
            if z not in U:
                U.add(z)
                S.append(add_to_pset(z, x, h))

    for z in maximal_parents(V, tau, _pgroups):
        if z not in U:
            S.append(z)

    return S


In [ ]:
psets = maximal_parents(tuple(range(n)), 1000)
len(psets)


1576


In [ ]:
%timeit maximal_parents(tuple(range(n)), 10000)

1.12 s ± 4.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%lprun -f maximal_parents maximal_parents(tuple(range(n)), 10000)

Timer unit: 1e-06 s

Total time: 8.72521 s
File: /tmp/ipykernel_2349234/3433675883.py
Function: maximal_parents at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def maximal_parents(V: tuple[int], tau: float, _pgroups=set()) -> list[tuple[int]]:
     2   1246985     553492.0      0.4      6.3      if tau < 1:
     3    307524     117733.0      0.4      1.3          return []
     4    939461     365033.0      0.4      4.2      if not V:
     5    324176     132790.0      0.4      1.5          return [empty_pset]
     6    615285     256888.0      0.4      2.9      x = V[0]
     7    615285     296507.0      0.5      3.4      V = V[1:]
     8                                           
     9    615285     235346.0      0.4      2.7      S = []
    10    615285     273404.0      0.4      3.1      U = set()
    11                                           
    12   1246984     628877.0      0.5      7.2      for h in